### imports

In [67]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
%autoreload 2

In [69]:
from regex_to_NFA import NFA_CLASS,NFA
from NFA_to_DFA import DFA_CLASS,DfaState,DfaEdge
import json
from parser_classes import EPSILON

### Functions

In [70]:
def json_to_nfa(json_path: str) -> NFA:
    """
    This function is used to convert the JSON data to NFA
    params:
        json_path: str: Path to the JSON file
    return:
        NFA: NFA object
    """
    # read the JSON file
    with open(json_path, "r") as file:
        json_data = json.load(file)
    # create the NFA object
    nfa = NFA(None, None, [], [])
    # create the states
    states = []
    acceping_index = 0
    for state in json_data:
        if state == "startingState":
            continue
        if json_data[state]["isTerminatingState"] == True:
            acceping_index = int(state[1:])
        states.append(DfaState(name=state))
    nfa.states = states
    # set the start state becuase the name of the starting states is the index of it inside the states array
    nfa.start = states[int(json_data["startingState"][1:])]
    # set the accept state
    nfa.accept = states[acceping_index]
    # create the transitions
    transitions = []
    for state in json_data:
        if state == "startingState":
            continue
        for transition in json_data[state]:
            if transition == "isTerminatingState":
                continue

            # check if the transition is epsilon or not
            if transition.startswith("epsilon"):
                transitions.append(
                    DfaEdge(
                        states[int(state[1:])],
                        states[int(json_data[state][transition][1:])],
                        {EPSILON},
                    )
                )
            else:
                transitions.append(
                    DfaEdge(
                        states[int(state[1:])],
                        states[int(json_data[state][transition][1:])],
                        {transition},
                    )
                )
    nfa.transitions = transitions
    return nfa

In [71]:
# getting the inputs from the tansitions in the nfa
def get_inputs(transitions):
    """_summary_
    This function is used to get the inputs from the transitions in the NFA
    Args:
        transitions (List): List of transitions in the NFA (List of Edges)
    Returns:
        List: List of unique inputs in the NFA
    """
    inputs = []
    for transition in transitions:
        temp_in = list(transition.characters)[0]
        if temp_in not in inputs and temp_in != EPSILON:
            inputs.append(temp_in)
    return inputs

### Testing

In [72]:
# input_regex = input("Enter the regular expression: ")
input_regex = "[a-z]*[0-9]+"
nfa = NFA_CLASS(input_regex)
deserialized_nfa = json_to_nfa("./nfa.json")
inputs = get_inputs(deserialized_nfa.transitions)
dfa = DFA_CLASS()
dfa.subset_construction(deserialized_nfa, inputs)
dfa.rename_dfa_states()
dfa.dfa_to_json("dfa.json")
dfa.visualize_dfa()
# dfa.print_dfa()
minimized_dfa = dfa.minimize_dfa(inputs)
minimized_dfa_obj = DFA_CLASS(minimized_dfa,inputs)
minimized_dfa_obj.print_dfa()
minimized_dfa_obj.dfa_to_json("minimized_dfa.json")
minimized_dfa_obj.visualize_dfa(path="minimized_dfa")

Visualization of the NFA is saved in nfa
Visualization of the DFA is saved in ./dfa
Final Partition:  [{DfaState(name='S2')}, {DfaState(name='S1'), DfaState(name='S0')}]
Start State:  S1
Accepting States:  ['S0']
Non Accepting States:  ['S1']
States:  ['S0', 'S1']
Transitions: 
From: S1, To: S1, Characters: {'a-z'}
From: S1, To: S0, Characters: {'0-9'}
From: S1, To: S1, Characters: {'a-z'}
From: S1, To: S0, Characters: {'0-9'}
From: S0, To: S0, Characters: {'0-9'}
Visualization of the DFA is saved in minimized_dfa
